# 📔 Jupyter Notebook: Construyendo un Sistema RAG con Python


## Objetivo

En este notebook, vamos a construir un sistema de **Retrieval-Augmented Generation (RAG)** desde sus fundamentos. El objetivo es entender cada componente del proceso sin depender de librerías de alto nivel como LangChain o LlamaIndex. Usaremos Python, una base de datos SQLite para actuar como nuestra base de conocimiento vectorial, y las APIs de **OpenAI, Google Gemini y Cohere** para la generación de embeddings y las respuestas finales.

Este enfoque nos permitirá ver "debajo del capó" cómo funciona RAG, un conocimiento crucial para cualquiera que quiera especializarse en la creación de aplicaciones con Modelos de Lenguaje Grandes (LLMs).

## 🧐 ¿Qué es RAG y por qué es importante?

Un LLM, como GPT-4, Gemini o Command R+, tiene un conocimiento inmenso, pero limitado a la fecha en que fue entrenado. Además, no conoce tus datos privados o información muy específica y reciente. Esto genera dos problemas principales:

1.  **Desconocimiento de datos específicos:** No puede responder preguntas sobre tus documentos internos, tu base de código o eventos ocurridos después de su fecha de corte de conocimiento.
2.  **Alucinaciones:** Cuando no sabe una respuesta, a veces puede "inventar" información que suena plausible pero es incorrecta.

**RAG soluciona esto** conectando el LLM a una base de conocimiento externa. El proceso, en pocas palabras, es:

1.  **Retrieval (Recuperación):** Ante una pregunta del usuario, el sistema primero busca información relevante en una base de datos (nuestra fuente de conocimiento).
2.  **Augmentation (Aumentación):** La información recuperada se añade al prompt original del usuario como "contexto".
3.  **Generation (Generación):** Se envía este prompt "aumentado" al LLM, que ahora tiene la información necesaria para generar una respuesta precisa y fundamentada en los datos proporcionados.

![Diagrama RAG](https://datos.gob.es/sites/default/files/datosgobes/generacion-aumentada_1.jpg)

En nuestro caso, la "base de conocimiento" será una colección de funciones de Python, y construiremos un sistema que pueda responder preguntas sobre cómo funcionan.

### **Paso 0: Instalación y Configuración de Dependencias**

Primero, necesitamos instalar las librerías necesarias.

In [ ]:
# Descomenta y ejecuta la siguiente línea si no tienes instaladas las librerías
# !pip install openai google-generativeai cohere numpy scikit-learn

A continuación, importaremos las librerías y, lo más importante, configuraremos las claves de API.

**⚠️ ¡Importante!** Nunca escribas tus claves de API directamente en el código. Usa variables de entorno o un gestor de secretos. Por simplicidad en esta clase, las cargaremos desde variables que debes definir.

In [ ]:
import os
import sqlite3
import numpy as np
import json
import openai
import google.generativeai as genai
import cohere

# --- CONFIGURACIÓN DE LAS API KEYS ---
# Asegúrate de tener tus claves de API como variables de entorno
# o reemplaza "YOUR_API_KEY" con tu clave correspondiente.

# OpenAI
#os.environ["OPENAI_API_KEY"] = "sk-..." # Reemplaza con tu clave de OpenAI
openai.api_key = os.getenv("OPENAI_API_KEY")

# Google Gemini
#os.environ["GEMINI_API_KEY"] = "AIza..." # Reemplaza con tu clave de Google
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

# Cohere
#os.environ["COHERE_API_KEY"] = "..." # Reemplaza con tu clave de Cohere
co = cohere.Client(os.getenv("COHERE_API_KEY"))

print("Librerías importadas y APIs configuradas.")

### **Paso 1: Nuestra Base de Conocimiento (El Código)**

Para este ejemplo, nuestra base de conocimiento será una colección de funciones de Python. En un caso real, esto podría ser una gran base de código, documentación o cualquier conjunto de textos.

In [ ]:
# Nuestra base de conocimiento: una lista de funciones de Python como strings.
# Cada función será un "documento" individual en nuestra base de datos.
code_snippets = [
    """
def calcular_fibonacci(n):
    \"\"\"Calcula el n-ésimo número de la secuencia de Fibonacci de forma recursiva.\"\"\"
    if n <= 0:
        return 0
    elif n == 1:
        return 1
    else:
        return calcular_fibonacci(n-1) + calcular_fibonacci(n-2)
    """,
    """
def es_palindromo(cadena):
    \"\"\"Verifica si una cadena de texto es un palíndromo.\"\"\"
    cadena = cadena.lower().replace(' ', '')
    return cadena == cadena[::-1]
    """,
    """
def quicksort(arr):
    \"\"\"Implementa el algoritmo de ordenamiento Quicksort.\"\"\"
    if len(arr) <= 1:
        return arr
    else:
        pivot = arr[len(arr) // 2]
        left = [x for x in arr if x < pivot]
        middle = [x for x in arr if x == pivot]
        right = [x for x in arr if x > pivot]
        return quicksort(left) + middle + quicksort(right)
    """,
    """
def buscar_en_lista_ordenada(lista, elemento):
    \"\"\"Realiza una búsqueda binaria en una lista ordenada.\"\"\"
    inicio, fin = 0, len(lista) - 1
    while inicio <= fin:
        medio = (inicio + fin) // 2
        if lista[medio] == elemento:
            return medio
        elif lista[medio] < elemento:
            inicio = medio + 1
        else:
            fin = medio - 1
    return -1 # Elemento no encontrado
    """
]

print(f"Tenemos {len(code_snippets)} fragmentos de código en nuestra base de conocimiento.")

### **Paso 2: Chunking (Fragmentación)**

El "chunking" es el proceso de dividir grandes textos en fragmentos más pequeños o "chunks". ¿Por qué hacemos esto?

1.  **Límite del Contexto:** Los modelos de embedding tienen un límite en la cantidad de texto que pueden procesar a la vez.
2.  **Relevancia:** Fragmentos más pequeños y enfocados suelen ser más relevantes para una consulta específica que un documento entero, lo que mejora la calidad de la búsqueda.

En nuestro caso, hemos hecho un "chunking manual": cada función es un chunk. Esto es una estrategia válida cuando los documentos son naturalmente modulares.

### **Paso 3: Generación de Embeddings**

Un **embedding** es una representación vectorial (una lista de números) de un fragmento de texto. La magia de los embeddings es que capturan el *significado semántico* del texto. Textos con significados similares tendrán vectores que "apuntan" en direcciones parecidas en un espacio multidimensional.

Crearemos funciones para generar embeddings usando las tres APIs.

In [ ]:
# Diccionario para mapear nombres de modelos a sus dimensiones
# Esto es útil para crear la base de datos correctamente.
EMBEDDING_MODELS = {
    "openai": {"model": "text-embedding-3-small", "dimensions": 1536},
    "gemini": {"model": "models/text-embedding-004", "dimensions": 768},
    "cohere": {"model": "embed-multilingual-v3.0", "dimensions": 1024}
}


def get_openai_embedding(text, model="text-embedding-3-small"):
    """Genera embeddings usando la API de OpenAI."""
    text = text.replace("\n", " ")
    response = openai.embeddings.create(input=[text], model=model)
    return response.data[0].embedding

def get_gemini_embedding(text, model="models/text-embedding-004"):
    """Genera embeddings usando la API de Google Gemini."""
    return genai.embed_content(model=model, content=text)["embedding"]

def get_cohere_embedding(text, model="embed-multilingual-v3.0"):
    """Genera embeddings usando la API de Cohere."""
    response = co.embed(texts=[text], model=model)
    return response.embeddings[0]

# Probemos una de las funciones
ejemplo_embedding = get_gemini_embedding("Hola mundo")
print("Ejemplo de embedding con Gemini:")
print(f"Tipo: {type(ejemplo_embedding)}")
print(f"Longitud (dimensiones): {len(ejemplo_embedding)}")
print(f"Primeros 5 valores: {ejemplo_embedding[:5]}")

### **Paso 4: Almacenamiento en SQLite**

Ahora, necesitamos un lugar para guardar nuestros chunks de código y sus correspondientes embeddings. Usaremos **SQLite**, una base de datos ligera y sin servidor que guarda todo en un único fichero. Es perfecta para proyectos pequeños y educativos.

Crearemos una tabla para almacenar:
* `id`: Identificador único.
* `model_name`: El modelo que generó el embedding (ej. 'openai').
* `chunk`: El fragmento de código.
* `embedding`: El vector numérico. Como SQLite no tiene un tipo de dato "array", lo guardaremos como un `BLOB` (Binary Large Object) tras serializarlo.

In [ ]:
DB_FILE = "rag_database.db"

def setup_database():
    """Crea la base de datos y la tabla si no existen."""
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS code_embeddings (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            model_name TEXT NOT NULL,
            chunk TEXT NOT NULL,
            embedding BLOB NOT NULL
        )
    """)
    # Limpiamos la tabla para ejecuciones frescas en este notebook
    cursor.execute("DELETE FROM code_embeddings")
    conn.commit()
    conn.close()
    print("Base de datos y tabla 'code_embeddings' creadas y limpiadas.")

def store_embeddings(model_name, chunks, embeddings):
    """Almacena los chunks y sus embeddings en la base de datos."""
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    for chunk, embedding in zip(chunks, embeddings):
        # Serializamos el embedding (array de numpy) a bytes para guardarlo como BLOB
        embedding_blob = np.array(embedding, dtype=np.float32).tobytes()
        cursor.execute(
            "INSERT INTO code_embeddings (model_name, chunk, embedding) VALUES (?, ?, ?)",
            (model_name, chunk, embedding_blob)
        )
    conn.commit()
    conn.close()

# --- Proceso Principal de Población de la Base de Datos ---

setup_database()

# Elegimos qué modelo usar para poblar la base de datos.
# Podríamos hacerlo para los tres, pero para la demo, usemos uno.
chosen_model_name = "gemini" # Puedes cambiar a "openai" o "cohere"

print(f"\nGenerando y almacenando embeddings usando el modelo: {chosen_model_name}...")

embedding_function = None
if chosen_model_name == "openai":
    embedding_function = get_openai_embedding
elif chosen_model_name == "gemini":
    embedding_function = get_gemini_embedding
elif chosen_model_name == "cohere":
    embedding_function = get_cohere_embedding

# Generamos los embeddings para cada fragmento de código
embeddings_to_store = [embedding_function(chunk) for chunk in code_snippets]

# Los almacenamos en la base de datos
store_embeddings(chosen_model_name, code_snippets, embeddings_to_store)

print(f"¡Éxito! Se han almacenado {len(code_snippets)} chunks y sus embeddings en '{DB_FILE}'.")

### **Paso 5: Retrieval (Recuperación de Información)**

Esta es la parte "R" de RAG. El proceso es:
1.  Tomar la pregunta del usuario (la "query").
2.  Generar un embedding para esa query **usando el mismo modelo que usamos para crear los embeddings de la base de datos**. ¡Esto es crucial!
3.  Recuperar todos los embeddings de la base de datos.
4.  Calcular la similitud entre el embedding de la query y todos los embeddings almacenados.
5.  Seleccionar los `k` fragmentos de código (chunks) cuyos embeddings son más similares.

La métrica de similitud más común es la **similitud del coseno**, que mide el coseno del ángulo entre dos vectores. Un valor de 1 significa que son idénticos en orientación, 0 que son ortogonales, y -1 que son opuestos.

La fórmula es:
$$\text{similitud}(\mathbf{A}, \mathbf{B}) = \frac{\mathbf{A} \cdot \mathbf{B}}{\|\mathbf{A}\| \|\mathbf{B}\|} = \frac{\sum_{i=1}^{n} A_i B_i}{\sqrt{\sum_{i=1}^{n} A_i^2} \sqrt{\sum_{i=1}^{n} B_i^2}}$$

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def retrieve_top_k(query, model_name, top_k=2):
    """Recupera los k chunks más relevantes de la base de datos."""
    print(f"Buscando los {top_k} chunks más relevantes para la query usando el modelo '{model_name}'...")

    # 1. Generar embedding para la query
    embedding_function = None
    if model_name == "openai":
        embedding_function = get_openai_embedding
    elif model_name == "gemini":
        embedding_function = get_gemini_embedding
    elif model_name == "cohere":
        embedding_function = get_cohere_embedding

    query_embedding = embedding_function(query)
    query_embedding_np = np.array(query_embedding).reshape(1, -1) # Convertir a array 2D

    # 2. Obtener todos los chunks y embeddings de la BD
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    cursor.execute("SELECT chunk, embedding FROM code_embeddings WHERE model_name = ?", (model_name,))
    db_results = cursor.fetchall()
    conn.close()

    if not db_results:
        print("No se encontraron embeddings en la base de datos para el modelo especificado.")
        return []

    db_chunks = [row[0] for row in db_results]
    
    # Deserializar los embeddings de BLOB a numpy array
    embedding_dim = EMBEDDING_MODELS[model_name]["dimensions"]
    db_embeddings = [np.frombuffer(row[1], dtype=np.float32) for row in db_results]
    db_embeddings_np = np.array(db_embeddings)

    # 3. Calcular la similitud del coseno
    similarities = cosine_similarity(query_embedding_np, db_embeddings_np)[0]

    # 4. Encontrar los top-k
    # Obtenemos los índices de las mayores similitudes en orden descendente
    top_k_indices = np.argsort(similarities)[::-1][:top_k]

    # 5. Devolver los chunks correspondientes
    relevant_chunks = [db_chunks[i] for i in top_k_indices]
    
    print("Chunks recuperados con éxito.")
    return relevant_chunks

# --- Prueba de Recuperación ---
user_query = "¿cómo puedo ordenar una lista de números?"
# Usamos el mismo modelo con el que poblamos la BD
retrieved_chunks = retrieve_top_k(user_query, chosen_model_name, top_k=1)

print("\n--- Contexto Recuperado ---")
for i, chunk in enumerate(retrieved_chunks):
    print(f"Chunk {i+1}:\n{chunk}\n")

Como puedes ver, el sistema recuperó correctamente la función `quicksort` como el contexto más relevante para la pregunta sobre "ordenar una lista". ¡El retrieval funciona!

### **Paso 6: Augmentation y Generation (Aumentación y Generación)**

Ahora que tenemos el contexto relevante, pasamos a la parte "G" de RAG.

1.  **Aumentación:** Creamos un nuevo prompt para el LLM. Este prompt incluirá el contexto recuperado y la pregunta original del usuario. Un buen formato es clave para que el modelo entienda qué hacer.
2.  **Generación:** Enviamos este prompt aumentado a un LLM (puede ser de OpenAI, Gemini o Cohere) para que genere la respuesta final.

In [ ]:
def build_prompt(query, context_chunks):
    """Construye el prompt aumentado para el LLM."""
    context = "\n\n---\n\n".join(context_chunks)
    
    prompt = f"""
Eres un asistente experto en Python. Responde a la pregunta del usuario basándote únicamente en el siguiente contexto de código.
Si el contexto no contiene la respuesta, di que no tienes suficiente información.

Contexto:
{context}

Pregunta del usuario:
{query}

Respuesta:
"""
    return prompt

def get_llm_response(prompt, provider="gemini"):
    """Obtiene una respuesta de un LLM dado un prompt."""
    print(f"\nEnviando prompt aumentado a {provider.upper()}...")
    try:
        if provider == "openai":
            response = openai.chat.completions.create(
                model="gpt-4o-mini",
                messages=[{"role": "user", "content": prompt}]
            )
            return response.choices[0].message.content
        elif provider == "gemini":
            model = genai.GenerativeModel('gemini-2.5-flash')
            response = model.generate_content(prompt)
            return response.text
        elif provider == "cohere":
            response = co.chat(
                model='command-r',
                message=prompt
            )
            return response.text
    except Exception as e:
        return f"Error al contactar la API de {provider}: {e}"


# --- Proceso Final: Poniéndolo Todo Junto ---

# 1. Definimos la query del usuario
final_user_query = "¿Cuál es un buen algoritmo para ordenar una lista y cómo funciona?"

# 2. Recuperamos el contexto (Retrieval)
# Usamos el modelo con el que creamos la BD.
context_chunks = retrieve_top_k(final_user_query, chosen_model_name, top_k=1)

# 3. Construimos el prompt (Augmentation)
final_prompt = build_prompt(final_user_query, context_chunks)

print("\n--- Prompt Final Enviado al LLM ---")
print(final_prompt)

# 4. Generamos la respuesta final (Generation)
# Vamos a probar con los tres modelos para comparar

# Respuesta con Gemini
response_gemini = get_llm_response(final_prompt, provider="gemini")
print("\n--- Respuesta de Gemini ---")
print(response_gemini)

# Respuesta con OpenAI
response_openai = get_llm_response(final_prompt, provider="openai")
print("\n--- Respuesta de OpenAI ---")
print(response_openai)

# Respuesta con Cohere
response_cohere = get_llm_response(final_prompt, provider="cohere")
print("\n--- Respuesta de Cohere ---")
print(response_cohere)

## Conclusiones y Próximos Pasos

¡Felicidades! Has construido un sistema RAG completo desde cero.

**Hemos aprendido a:**
1.  **Fragmentar (Chunking):** Dividir el conocimiento en piezas manejables.
2.  **Generar Embeddings:** Convertir texto en vectores semánticos usando APIs de vanguardia.
3.  **Almacenar:** Guardar estos datos en una base de datos simple como SQLite.
4.  **Recuperar (Retrieval):** Implementar una búsqueda por similitud de coseno para encontrar los chunks más relevantes para una query.
5.  **Aumentar y Generar (Augmented Generation):** Construir un prompt con contexto y usar un LLM para obtener una respuesta fundamentada.

**Para explorar más allá:**
* **Vector Databases:** En lugar de SQLite y cálculo manual, investiga bases de datos vectoriales dedicadas como **ChromaDB, FAISS o Pinecone**, que están optimizadas para búsquedas de similitud a gran escala.
* **Estrategias de Chunking Avanzadas:** Investiga el chunking semántico o el chunking basado en tokens con solapamiento (`overlapping`).
* **Re-ranking:** A veces, los `k` mejores resultados no son los más relevantes. Un paso de "re-ranking" con un modelo más sofisticado (como Cohere Rerank) puede mejorar aún más la selección de contexto.
* **Interfaz Gráfica:** ¡Convierte este notebook en una pequeña aplicación web usando **Streamlit** o **Gradio**!